In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
from time import perf_counter
import glob
import json
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.pyplot import figure
import tweets_utils
import importlib
%autosave 3600

Autosaving every 3600 seconds


# Indice

- [General opening files](#opening)
- [Wrong situation](#wrong)

### General opening files
<a id="opening"/>

In [2]:
files_tweets = glob.glob("F:/COVID-19-Tweets/*/tweets.csv")

In [3]:
chunksize = 10
for chunk in pd.read_csv(files_tweets[0], chunksize=10, lineterminator = '\n'):
    df = chunk
    break
df

,id,id_str,created_at,user_id,user_screen_name,user_verified,user_created_at,text,source,coordinates,...,quoted_status_text,quoted_status_created_at,quoted_user_id,quoted_user_created_at,hashtags,symbols,user_mentions,urls,polls,media
0,1219752899636613121,1219752899636613121,Tue Jan 21 22:45:27 +0000 2020,1110906564158869505,Huerconetzin,False,Wed Mar 27 14:08:59 +0000 2019,@CDC has activated its emergency operations ce...,"<a href=""http://twitter.com/download/android"" ...",NaN,...,NaN,NaN,NaN,NaN,"[{'text': 'coronavirus', 'indices': [94, 106]}]",[],"[{'screen_name': 'AnneKPIX', 'name': 'Anne Mak...",[],[],[]
1,1219768733310291969,1219768733310291969,Tue Jan 21 23:48:22 +0000 2020,1214557289606242304,JustAnotherAme4,False,Tue Jan 07 14:40:19 +0000 2020,"Soon, passengers from Wuhan to the US, on dire...","<a href=""http://twitter.com/download/android"" ...",NaN,...,NaN,NaN,NaN,NaN,[],[],"[{'screen_name': 'cnni', 'name': 'CNN Internat...",[],[],[]
2,1219768903116607488,1219768903116607488,Tue Jan 21 23:49:03 +0000 2020,426033838,HHSRegion8,True,Thu Dec 01 19:14:42 +0000 2011,The first human infection with new #coronaviru...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,"[{'text': 'coronavirus', 'indices': [47, 59]}]",[],"[{'screen_name': 'CDCgov', 'name': 'CDC', 'id'...",[],[],[]
3,1219753863940648965,1219753863940648965,Tue Jan 21 22:49:17 +0000 2020,2252416118,Paxman42,False,Wed Dec 18 19:04:31 +0000 2013,Where is our FEDERAL vaccination education cam...,"<a href=""http://twitter.com/download/android"" ...",NaN,...,NaN,NaN,NaN,NaN,"[{'text': 'lungcancer', 'indices': [103, 114]}]",[],"[{'screen_name': 'OurWarOnCancer', 'name': 'Ou...",[],[],[]
4,1219755288988798981,1219755288988798981,Tue Jan 21 22:54:57 +0000 2020,738000529775697920,beerhowell,False,Wed Jun 01 13:33:32 +0000 2016,Breaking News: The first U.S. case of the Wuha...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,[],[],"[{'screen_name': 'nytimes', 'name': 'The New Y...",[],[],[]
5,1219771346768596992,1219771346768596992,Tue Jan 21 23:58:45 +0000 2020,1136370591517290496,Gambiste1,False,Wed Jun 05 20:33:56 +0000 2019,$SPY $AAL $APT $EXPE $GOOG $DAL $UAL $BA $LAK...,"<a href=""https://gambiste.com"" rel=""nofollow"">...",NaN,...,NaN,NaN,NaN,NaN,"[{'text': 'CDC', 'indices': [89, 93]}, {'text'...","[{'text': 'SPY', 'indices': [20, 24]}, {'text'...","[{'screen_name': 'StocksUnhinged', 'name': 'St...",[],[],[]
6,1219753292571594754,1219753292571594754,Tue Jan 21 22:47:01 +0000 2020,33833239,IAmTonyaNash,False,Tue Apr 21 06:27:31 +0000 2009,PHAP gives future #publichealth professionals ...,"<a href=""http://twitter.com/#!/download/ipad"" ...",NaN,...,NaN,NaN,NaN,NaN,"[{'text': 'publichealth', 'indices': [30, 43]}]",[],"[{'screen_name': 'CDCgov', 'name': 'CDC', 'id'...","[{'url': 'https://t.co/8kzRD1IADl', 'expanded_...",[],[]
7,1219769722171994112,1219769722171994112,Tue Jan 21 23:52:18 +0000 2020,545294451,AndiUgo,False,Wed Apr 04 15:27:00 +0000 2012,CDC announces first U.S. case of Chinese coron...,"<a href=""http://www.facebook.com/twitter"" rel=...",NaN,...,NaN,NaN,NaN,NaN,[],[],[],"[{'url': 'https://t.co/2u7F1xZZYM', 'expanded_...",[],[]
8,1219755883690774529,1219755883690774529,Tue Jan 21 22:57:19 +0000 2020,2606294286,_ayychris,False,Sat Jul 05 23:15:29 +0000 2014,Breaking: The CDC is expected to announce the ...,"<a href=""http://twitter.com/download/android"" ...",NaN,...,NaN,NaN,NaN,NaN,[],[],"[{'screen_name': 'PMBreakingNews', 'name': 'PM...",[],[],[]
9,1219754232498266112,1219754232498266112,Tue Jan 21 22:50:45 +0000 2020,16794148,ParkvilleMOm,False,Wed Oct 15 20:57:08 +0000 2008,The U.S. Centers for Disease Control (CDC) con...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,[],[],"[{'screen_name': 'Reuters', 'name': 'Reuters',...",[],[],[]


In [4]:
path_hashtag = glob.glob("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/csv/")
hashtag_disinform = pd.read_csv(path_hashtag[0] + "hashtags_sus.csv")
hashtag_disinform

,hashtag
0,TheDefender
1,Trump
2,Misinformation
3,BillGates
4,sars
5,Monsanto
6,TrumpONEaz
7,Ebola
8,Fauci


In [5]:
lista_hashtags = list(hashtag_disinform["hashtag"])
lista_hashtags
#TheDefender --> NOVAX 
#BillGates --> novax
#Monsanto --> no ogm e big pharma
#TrumpONEaz --> no info
#Misinformation, sars, Ebola, Trump, Fauci --> Troppo variegati

['TheDefender',
 'Trump',
 'Misinformation',
 'BillGates',
 'Monsanto',
 'TrumpONEaz',
 'Fauci']

In [6]:
cols_tweets = [u'user_id',u'user_screen_name',u'hashtags'] 
users = []
hashtags = []

In [7]:
result = []
importlib.reload(tweets_utils)
start_time = time.perf_counter()
j = 0
data = pd.DataFrame()
for filename in files_tweets:
    print(f"Reading file: {filename}")
    result.extend(tweets_utils.process_all_data(filename, cols_tweets, True))
    lista = result[j]
    j = j + 1
    print(f"Mese numero {j}")
    for i in range(len(lista)):
        users.extend(lista[i]["name"])
        hashtags.extend(lista[i]["hashtags"])    
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

Reading file: F:/COVID-19-Tweets\2020-01\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Mese numero 1
Reading file: F:/COVID-19-Tweets\2020-02\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Mese numero 2
Reading file: F:/COVID-19-Tweets\2020-03\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Mese numero 3
Reading file: F:/COVID-19-Tweets\2020-04\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7


In [32]:
del result
del lista

### Wrong situation
<a id="wrong"/>

In [8]:
a = ""
for i in lista_hashtags:
    a = i
    break
#     a = i + "|"
# a = a[0: -1]
a

'TheDefender'

In [9]:
series_hashtags = pd.Series(hashtags)
del hashtags

In [10]:
name_user = []
hashtag_used = []

In [11]:
start_time = time.perf_counter()
val = (series_hashtags).str.contains(a)
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

Time:  94.88352940000004


In [12]:
start_time = time.perf_counter()
for i in range(len(val)):
    if(i % 10000000 == 0):
        print(i)
    if val[i] == True:
        hashtag_used.append(series_hashtags[i])
        name_user.append(users[i])
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

0
10000000
20000000
30000000
40000000
50000000
60000000
Time:  619.0385501999999


In [ ]:
del series_hashtags
del users

In [16]:
name_user

['knastergast',
 'SotoAlexisEd',
 'LlewelynPritch',
 'Gigakoos',
 'NvrKnew1',
 'ColeenChase',
 'Kaka_Laux',
 'STEM2Crisis',
 'camancher2012',
 'Seidenstrasse',
 'GilbertRossier',
 'SaribuhayUPD',
 'LeonDulce_',
 'aramarie001',
 'terricolaruben',
 'ChesleighNofiel',
 'manueloesch',
 'Kl_Stone',
 'nrglass37']

In [15]:
len(hashtag_used)

19

In [18]:
hashtag_ = [tweets_utils.hashtag_normalize(x) for x in hashtag_used]

In [19]:
hashtag_

[['DefendTheDefenders'],
 ['DefendTheDefenders'],
 ['DefendTheDefenders'],
 ['DefendTheDefenders'],
 ['Brazil',
  'SaoPaulo',
  'Amazonia',
  'DefendTheDefenders',
  'Democracy',
  'Journalism',
  'News',
  'ClimateChange',
  'Covid19',
  'PoliticsToday',
  'Politics'],
 ['climatecrisis', 'DefendTheDefenders'],
 ['NoForestNoFuture',
  'DefendTheDefenders',
  'ClimateStrikeOnline',
  'RacialJustice',
  'BlackLivesMatter',
  'IndianLivesMatter',
  'FridaysForFuture',
  'StayHome'],
 ['COVID19',
  'Crisis',
  'DefendTheDefenders',
  'FightEveryCrisis',
  'FridaysForFuture',
  'ClimateStrikeOnline',
  'Nurses'],
 ['conservation', 'nature', 'DefendTheDefenders'],
 ['COVID19', 'StopTheAttacks', 'DefendTheDefenders'],
 ['StopDatteln4',
  'ENDCOAL',
  'StopCareDrainers',
  'LeaveNoOneBehind',
  'FightEveryCrisis',
  'GreenRecoveryNow',
  'DefendTheDefenders',
  'ClimateJustice'],
 ['COVID19', 'StopTheAttacks', 'DefendTheDefenders'],
 ['COVID19', 'GreenNewNormal', 'StopTheAttacks', 'DefendTheDe